# Change-Point Analysis

This notebook demonstrates automated stratigraphic interpretation using GeoSuite's change-point detection functions.

## Overview

Change-point detection is used to automatically identify formation boundaries in well logs. GeoSuite provides:
- **PELT algorithm**: Optimal segmentation with penalty tuning
- **Bayesian online detection**: Probabilistic change-point detection
- **Preprocessing**: Median filtering and baseline removal
- **Consensus picking**: Combine multiple methods for robust results

This notebook will show you how to:

1. Load and preprocess well log data
2. Apply change-point detection algorithms
3. Compare multiple methods
4. Generate consensus formation tops

In [ ]:
# Import GeoSuite modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from geosuite.data import load_demo_well_logs
    preprocess_log,
    detect_pelt,
    detect_bayesian_online,
    compare_methods,
    find_consensus,
    tune_penalty_to_target_count
)

print("GeoSuite imported successfully!")

## 1. Load and Prepare Well Log Data

Load well log data and extract a log curve (e.g., gamma ray) for change-point detection.

In [ ]:
# Load demo well log data
df = load_demo_well_logs()

print(f"Loaded {len(df):,} data points")
print(f"\nAvailable columns: {df.columns.tolist()}")

# Find gamma ray or similar log for change-point detection
gr_cols = [col for col in df.columns if 'GR' in col.upper() or 'GAMMA' in col.upper()]
depth_col = 'depth_m' if 'depth_m' in df.columns else 'DEPTH'

if gr_cols:
    log_col = gr_cols[0]
    print(f"\nUsing {log_col} for change-point detection")
else:
    # Use first numeric column as fallback
    numeric_cols = [col for col in df.columns if col != depth_col and pd.api.types.is_numeric_dtype(df[col])]
    log_col = numeric_cols[0] if numeric_cols else df.columns[1]
    print(f"\nUsing {log_col} for change-point detection")

# Extract log values and depth
log_values = df[log_col].values
depth = df[depth_col].values if depth_col in df.columns else np.arange(len(df))

print(f"\nLog range: {log_values.min():.1f} to {log_values.max():.1f}")
print(f"Depth range: {depth.min():.1f} to {depth.max():.1f}")

## 2. Preprocess Log Data

Use `preprocess_log()` to remove spikes and drift while preserving formation boundaries.

In [ ]:
# Preprocess the log to remove spikes and drift
# Median filter removes spikes while preserving sharp edges
# Detrending removes long-wavelength drift while preserving bed-scale contrasts
log_processed = preprocess_log(
    log_values,
    median_window=5,      # Remove spikes (5 samples)
    detrend_window=100    # Remove drift (100 samples)
)

print(f"Original log: {len(log_values)} samples")
print(f"Processed log: {len(log_processed)} samples")
print(f"\nOriginal std: {np.std(log_values):.2f}")
print(f"Processed std: {np.std(log_processed):.2f}")

# Visualize preprocessing
fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(depth, log_values, 'lightgray', linewidth=0.5, label='Original', alpha=0.5)
ax.plot(depth, log_processed, 'black', linewidth=1, label='Processed')
ax.set_xlabel('Depth')
ax.set_ylabel(log_col)
ax.set_title('Log Preprocessing - Spike and Drift Removal')
ax.legend()
ax.invert_yaxis()  # Depth increases downward
plt.tight_layout()
plt.show()

## 3. Detect Change Points with PELT

Use `detect_pelt()` for optimal segmentation. PELT finds the global optimum efficiently.

In [ ]:
# Detect change points using PELT algorithm
try:
    # Auto-tune penalty for target pick density
    # Target: ~8 picks per 500 ft (adjust based on your stratigraphy)
    depth_increment = depth[1] - depth[0] if len(depth) > 1 else 0.5
    
    penalty = tune_penalty_to_target_count(
        log_processed,
        target_picks_per_500ft=8,
        depth_increment_ft=depth_increment
    )
    
    print(f"Tuned penalty: {penalty:.2f}")
    
    # Detect change points
    cp_indices = detect_pelt(log_processed, penalty=penalty)
    cp_depths = depth[cp_indices] if len(cp_indices) > 0 else np.array([])
    
    print(f"\nPELT detected {len(cp_indices)} change points")
    print(f"Formation tops (ft):")
    for i, d in enumerate(cp_depths[:10]):  # Show first 10
        print(f"  {i+1:2d}. {d:7.1f}")
    
except ImportError as e:
    print(f"PELT requires ruptures library: {e}")
    print("Install with: pip install ruptures")
    cp_indices = np.array([])
    cp_depths = np.array([])

## 4. Bayesian Online Change-Point Detection

Use `detect_bayesian_online()` for probabilistic change-point detection with uncertainty quantification.

In [ ]:
# Bayesian online change-point detection
# This method provides probability estimates for each change point
cp_bayes, cp_probs = detect_bayesian_online(
    log_processed,
    expected_segment_length=50.0,  # Expected length between changes (samples)
    threshold=0.5                   # Probability threshold for flagging changes
)

cp_bayes_depths = depth[cp_bayes] if len(cp_bayes) > 0 else np.array([])

print(f"Bayesian detection found {len(cp_bayes)} change points")
print(f"Max probability: {cp_probs.max():.3f}")
print(f"\nFormation tops (ft):")
for i, d in enumerate(cp_bayes_depths[:10]):
    prob = cp_probs[cp_bayes[i]] if i < len(cp_bayes) else 0.0
    print(f"  {i+1:2d}. {d:7.1f} (prob={prob:.3f})")

## 5. Compare Multiple Methods

Use `compare_methods()` to run multiple detection algorithms and compare results.

In [ ]:
# Compare multiple change-point detection methods
try:
    results = compare_methods(
        log_processed,
        depth,
        penalties=None,  # Auto-generate penalty range
        bayesian_threshold=0.5,
        include_kernel=True  # Include RBF kernel-based PELT
    )
    
    print(f"Ran {len(results)} detection methods:")
    for method_name, method_result in results.items():
        n_points = method_result.get('n_points', 0)
        print(f"  - {method_name}: {n_points} change points")
    
except Exception as e:
    print(f"Method comparison failed: {e}")
    results = {}

## 6. Find Consensus Picks

Use `find_consensus()` to combine results from multiple methods for robust formation tops.

In [ ]:
# Find consensus picks from multiple methods
if results:
    consensus = find_consensus(results, tolerance_ft=5.0)
    
    print(f"Consensus: {len(consensus)} formation tops")
    print(f"\nConsensus formation tops (ft):")
    for i, top_depth in enumerate(consensus):
        print(f"  {i+1:2d}. {top_depth:7.1f} ft")
    
    # Visualize consensus picks
    fig, ax = plt.subplots(figsize=(10, 8))
    ax.plot(depth, log_processed, 'black', linewidth=1, label='Processed Log')
    
    # Mark consensus picks
    for cp_depth in consensus:
        ax.axhline(y=cp_depth, color='red', linestyle='--', linewidth=1.5, alpha=0.7)
    
    ax.set_xlabel(log_col)
    ax.set_ylabel('Depth')
    ax.set_title(f'Change-Point Detection - {len(consensus)} Consensus Picks')
    ax.legend()
    ax.invert_yaxis()
    plt.tight_layout()
    plt.show()
else:
    print("No results available for consensus picking")

## 7. Summary

This notebook demonstrated:

-  Preprocessing logs with `preprocess_log()` (spike removal, drift correction)
-  PELT change-point detection with `detect_pelt()` and penalty tuning
-  Bayesian online detection with `detect_bayesian_online()`
-  Comparing multiple methods with `compare_methods()`
-  Finding consensus picks with `find_consensus()`

### Key Functions Used

- `preprocess_log()`: Remove spikes and drift while preserving boundaries
- `detect_pelt()`: Optimal segmentation (requires ruptures library)
- `detect_bayesian_online()`: Probabilistic detection (Numba-accelerated)
- `tune_penalty_to_target_count()`: Auto-tune PELT penalty
- `compare_methods()`: Run multiple algorithms
- `find_consensus()`: Combine results for robust picks

### Next Steps

- Validate picks against core tops or mudlog markers
- Adjust penalty/threshold based on formation thickness
- Use multiple logs (GR, resistivity, density) for multi-log consensus
- Export picks to well database or interpretation software